In [35]:
from decimal import Decimal
from string import ascii_lowercase
import datetime
from collections import namedtuple

class TimeZone:
    def __init__(self, offset, name):
        self.name = name
        self.offset = offset
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise ValueError('Name must be a string.')
        if not value.strip():
            raise ValueError('Name must not be blank.')
        self._name = value.strip()
        
    @property
    def offset(self):
        return self._offset
    
    @offset.setter
    def offset(self, value):
        if not isinstance(value, datetime.timedelta):
            raise TypeError('Offset must be a datetime.timedelta object.')
        if value < datetime.timedelta(hours=-12, minutes=0) or value > datetime.timedelta(hours=14, minutes=0):
            raise ValueError('Offset must be between -12:00 and +14:00.')
        self._offset = value
        
    def __repr__(self):
        return f'TimeZone(offset={self.offset}, name={self.name})'
        
class BankAccount:
    _monthly_interest_rate = None
    _tran_id = 0
    
    _transaction_codes = {
        'deposit': 'D',
        'withdrawal': 'W',
        'interest': 'I',
        'rejected': 'X'
    }
    
    def __init__(self, accnt_no, f_name, l_name, tz_offset=None, balance=None):
        self.accnt_no = accnt_no
        self.f_name = f_name
        self.l_name = l_name
        self.tz_offset = tz_offset
        self._balance = Decimal(0.00)
        if balance:
            self.deposit(balance)

    @staticmethod
    def _validate_name(value, field_title):
        # If every character in the provided value (assuming it's a string) is not a
        # standard letter, a space, or an apostrophe, raise an exception.
        if len(str(value).strip()) == 0 or value is None:
            raise ValueError(f'{field_title} must not be blank.')
        if not all(map(lambda x: x.lower() in ascii_lowercase + " '", str(value))):
            raise ValueError(f'{field_title} must contain only letters, spaces, or apostrophes.')
        return str(value).strip()
            
    @staticmethod
    def parse_conf_no(conf_no, timezone):
        ConfirmationNumber = namedtuple('ConfirmationNumber',
                                        ['account_number',
                                         'transaction_code',
                                         'transaction_id',
                                         'time',
                                         'time_utc']
                                       )
        
        conf_components = conf_no.split('-')
        tran_code = conf_components[0]
        accnt_no = conf_components[1]
        time_utc = datetime.datetime.strptime(conf_components[2], '%Y%m%d%H%M%S')
        tran_id = conf_components[3]
        time = f'{time_utc + timezone.offset} ({timezone.name})'
        
        parsed_conf_no = ConfirmationNumber(accnt_no,
                                            tran_code,
                                            tran_id,
                                            time,
                                            time_utc.isoformat())
        return parsed_conf_no
            
    @classmethod
    def get_interest_rate(cls):
        if cls._monthly_interest_rate is None:
            raise ValueError('Interest rate has not been set.')
        return cls._monthly_interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        try:
            value = Decimal(value)
        except:
            raise ValueError('Interest rate must be a decimal value.')
        if value < 0:
            raise ValueError('Interest rate cannot be negative.')
        cls._monthly_interest_rate = value
        return f'New monthly interest rate = {round(cls._monthly_interest_rate * 100, 2)}%'
    
    @classmethod
    def _generate_confirmation(cls, account_no, tran_type):
        cls._tran_id += 1
        return (
            f"{tran_type}-{account_no}-"
            f"{datetime.datetime.utcnow().strftime('%Y%m%d%H%M%S')}-"
            f"{cls._tran_id}"
        )
        
    @classmethod
    def _validate_transaction(cls, amount, accnt_no):
        try:
            amount = round(Decimal(amount), 2)
        except:
            conf_no = cls._generate_confirmation(accnt_no,
                                                  cls._transaction_codes['rejected'])
            print('Transaction amount must be a decimal value.')
            print('Confirmation number:', conf_no)
            return
        if amount <= 0:
            conf_no = cls._generate_confirmation(accnt_no,
                                                  cls._transaction_codes['rejected'])
            print('Transaction amount must be greater than zero.')
            print('Confirmation number:', conf_no)
            return
        else:
            return True
    
    @property
    def accnt_no(self):
        return self._accnt_no
    
    @accnt_no.setter
    def accnt_no(self, value):
        try:
            value = int(value)
        except:
            raise ValueError('Account number must be an integer value.')
        if value <= 0:
            raise ValueError('Account number must be greater than zero.')
        self._accnt_no = value
        
    @property
    def f_name(self):
        return self._f_name
    
    @f_name.setter
    def f_name(self, value):
        self._f_name = BankAccount._validate_name(value, 'First Name')
        
    @property
    def l_name(self):
        return self._l_name
    
    @l_name.setter
    def l_name(self, value):
        self._l_name = BankAccount._validate_name(value, 'Last Name')
        
    @property
    def full_name(self):
        return f'{self.f_name} {self.l_name}'
    
    @property
    def tz_offset(self):
        return self._tz_offset
    
    @tz_offset.setter
    def tz_offset(self, value):
        if value is None:
            value = TimeZone(datetime.timedelta(), 'UTC')
        if not isinstance(value, TimeZone):
            raise TypeError('tz_offset must be a TimeZone object.')
        self._tz_offset = value
        
    @property
    def balance(self):
        return round(self._balance, 2)
    
    def deposit(self, amount, interest=False):
        if self._validate_transaction(amount, self.accnt_no):
            self._balance += amount
            if interest:
                conf_no = self._generate_confirmation(self.accnt_no,
                                                  self._transaction_codes['interest'])
            else:
                conf_no = self._generate_confirmation(self.accnt_no,
                                                  self._transaction_codes['deposit'])
            print(
                "Deposit accepted.\n"
                f"New balance = {self.balance}\n"
                f"Confirmation number = {conf_no}"
            )
        
    def withdrawal(self, amount):
        if self._validate_transaction(amount, self.accnt_no):
            if self._balance - amount < 0:
                conf_no = self._generate_confirmation(self.accnt_no,
                                                      self._transaction_codes['rejected'])
                print('Insufficient funds.')
                print('Confirmation number:', conf_no)
                return

            self._balance -= amount
            conf_no = self._generate_confirmation(self.accnt_no,
                                                  self._transaction_codes['withdrawal'])
            print(
                "Withdrawal accepted.\n"
                f"New balance = {self.balance}\n"
                f"Confirmation number = {conf_no}"
            )
    
    def interest_deposit(self):
        deposit_amount = self.balance * self.get_interest_rate()
        return self.deposit(deposit_amount, interest=True)
    
    def __repr__(self):
        return (
            f"""BankAccount
    Account No: {self.accnt_no}
    Full Name: {self.full_name}
    Timezone: {self.tz_offset}
    Balance: {self.balance}"""
        )

In [36]:
ba = BankAccount(12345, 'Harry', 'Caray', balance=600)

Deposit accepted.
New balance = 600.00
Confirmation number = D-12345-20210307001317-1


In [37]:
ba

BankAccount
    Account No: 12345
    Full Name: Harry Caray
    Timezone: TimeZone(offset=0:00:00, name=UTC)
    Balance: 600.00

In [38]:
ba.withdrawal(45)

Withdrawal accepted.
New balance = 555.00
Confirmation number = W-12345-20210307001318-2


In [39]:
ba._validate_transaction('test', 12345)

Transaction amount must be a decimal value.
Confirmation number: X-12345-20210307001319-3


In [40]:
ba.deposit(200)

Deposit accepted.
New balance = 755.00
Confirmation number = D-12345-20210307001319-4


In [41]:
ba.set_interest_rate(0.005)

'New monthly interest rate = 0.50%'

In [42]:
ba.interest_deposit()

Deposit accepted.
New balance = 758.78
Confirmation number = I-12345-20210307001319-5


In [43]:
import unittest

In [44]:
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

In [63]:
class TestAccount(unittest.TestCase):
    def setUp(self):
        print('running setup...')
        self.x = 100
        
    def tearDown(self):
        print('running teardown...')
    
    def test_1(self):
        self.x = 200
        self.assertEqual(200, self.x)
        
    def test_2(self):
        self.assertEqual(200, self.x)

In [64]:

run_tests(TestAccount)

test_1 (__main__.TestAccount) ... ok
test_2 (__main__.TestAccount) ... 

running setup...
running teardown...
running setup...
running teardown...


FAIL

FAIL: test_2 (__main__.TestAccount)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-63-b48bc60ca1df>", line 14, in test_2
    self.assertEqual(200, self.x)
AssertionError: 200 != 100

----------------------------------------------------------------------
Ran 2 tests in 0.002s

FAILED (failures=1)


In [85]:
from decimal import Decimal
from string import ascii_lowercase
import datetime
from collections import namedtuple

class TestAccount(unittest.TestCase):
    
    def setUp(self):
        self.account_number = 12345
        self.first_name = 'FIRST'
        self.last_name = 'LAST'
        self.tz = TimeZone(datetime.timedelta(hours=-3, minutes=30), ' TZ ')
        self.balance = Decimal(100.00)
        
        self.a = BankAccount(self.account_number, self.first_name, self.last_name, self.tz, self.balance)
    
    def test_create_timezone(self):
        delta = datetime.timedelta(hours=-3, minutes=30)
        tz = TimeZone(delta, ' MST ')
        self.assertEqual('MST', tz.name)
        self.assertEqual(datetime.timedelta(hours=-3, minutes=30), tz.offset)
        
    def test_create_account(self):
        a = BankAccount(self.account_number, self.first_name, self.last_name, self.tz, self.balance)
        
        self.assertEqual(self.account_number, a.accnt_no)
        self.assertEqual(self.first_name, a.f_name)
        self.assertEqual(self.last_name, a.l_name)
        self.assertEqual(self.tz, a.tz_offset)
        self.assertEqual(self.balance, a.balance)
        
    def test_create_account_blank_first_name(self):
        self.first_name = ''
        
        with self.assertRaises(ValueError):
            a = BankAccount(self.account_number, self.first_name, self.last_name, self.tz, self.balance)
            
    def test_create_account_negative_balance(self):
        self.balance = -100
        
        a = BankAccount(self.account_number, self.first_name, self.last_name, self.tz, self.balance)
        
        self.assertEqual(0, a.balance)
        
    def test_account_withdrawal_ok(self):
        
        self.a.withdrawal(20)
        self.assertEqual(80, self.a.balance)
        
    def test_account_overdraw(self):
        
        self.a.withdrawal(200)
        self.assertEqual(100, self.a.balance)
        

In [86]:
run_tests(TestAccount)

test_account_overdraw (__main__.TestAccount) ... ok
test_account_withdrawal_ok (__main__.TestAccount) ... ok
test_create_account (__main__.TestAccount) ... ok
test_create_account_blank_first_name (__main__.TestAccount) ... ok
test_create_account_negative_balance (__main__.TestAccount) ... ok
test_create_timezone (__main__.TestAccount) ... 

Deposit accepted.
New balance = 100.00
Confirmation number = D-12345-20210307030757-23
Insufficient funds.
Confirmation number: X-12345-20210307030757-24
Deposit accepted.
New balance = 100.00
Confirmation number = D-12345-20210307030757-25
Withdrawal accepted.
New balance = 80.00
Confirmation number = W-12345-20210307030757-26
Deposit accepted.
New balance = 100.00
Confirmation number = D-12345-20210307030757-27
Deposit accepted.
New balance = 100.00
Confirmation number = D-12345-20210307030757-28
Deposit accepted.
New balance = 100.00
Confirmation number = D-12345-20210307030757-29
Deposit accepted.
New balance = 100.00
Confirmation number = D-12345-20210307030757-30
Transaction amount must be greater than zero.
Confirmation number: X-12345-20210307030757-31
Deposit accepted.
New balance = 100.00
Confirmation number = D-12345-20210307030757-32


ok

----------------------------------------------------------------------
Ran 6 tests in 0.005s

OK
